<a href="https://colab.research.google.com/github/pedromujica1/aqm-dataset/blob/main/4_TratamentoDadosEstacaoGrafana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# ========================
# [1] CARREGAR O DATASET
# ========================

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

from itertools import product


#Leitura da base de dados da estação grafana com os dados de CO
df_ae = pd.read_csv("co_ae.csv")
df_we = pd.read_csv("co_we.csv")
df_temp = pd.read_csv("temperatura.csv") #df da temperatura
df_bestCO = pd.read_csv('best-co.csv')

df_temp.columns = ['time', 'temperatura']
df_bestCO['time'] = pd.to_datetime(df_bestCO['time'], unit='ms')

# #padronizando time nas tabelas
df_bestCO['time'] = pd.to_datetime(df_bestCO['time'], unit='ms').dt.round('1s')
df_ae['time'] = pd.to_datetime(df_ae['time']).dt.round('1s')
df_we['time'] = pd.to_datetime(df_we['time']).dt.round('1s')
df_temp['time'] = pd.to_datetime(df_temp['time']).dt.round('1s')

# #Limpeza das colunas de AE e WE (remover " ppb" e converter para float)
df_ae['CO_AE'] = df_ae['CO_AE'].str.replace(' ppb', '', regex=False).astype(float)
df_we['CO_WE'] = df_we['CO_WE'].str.replace(' ppb', '', regex=False).astype(float)
df_temp['temperatura'] = df_temp['temperatura'].str.replace(' °C', '', regex=False) #eliminando strings

# #Mesclagem com base na coluna 'time'
df = df_bestCO.merge(df_ae, on='time', how='inner') \
              .merge(df_we, on='time', how='inner') \
              .merge(df_temp, on='time', how='inner')

df = df.dropna()
df = df.drop(columns=["time"])

#exportando para csb
df.to_csv("dados_grafana.csv", index=False)




In [12]:
from google.colab import files
uploaded = files.upload()

Saving modeloRF_Regressor.pickle to modeloRF_Regressor (1).pickle


In [13]:
import pickle
import pandas as pd

#carrega arquivo
with open('modeloRF_Regressor.pickle', 'rb') as f:
    modelo = pickle.load(f)


In [15]:


# [2] Carregar novos dados da estação (exemplo com mesmo formato)
novos_dados = pd.read_csv("dados_grafana.csv")  # nome do arquivo real

# [3] Selecionar as features
X_novos = novos_dados[["CO","CO_WE", "CO_AE", "temperatura"]]
X_novos




,CO,CO_WE,CO_AE,temperatura
0,929.284785,0.520,0.533,29.2
1,929.298999,0.520,0.531,29.2
2,929.331408,0.524,0.527,29.3
3,929.242062,0.517,0.540,29.4
4,929.162088,0.505,0.549,29.4
...,...,...,...,...
144216,1829.584450,0.538,0.546,23.9
144217,1829.584450,0.538,0.546,23.9
144218,1829.584450,0.538,0.546,23.9
144219,1829.584450,0.538,0.546,23.9


In [16]:
# [4] Fazer predição
y_pred = modelo.predict(X_novos)

# [5] Anexar ao DataFrame e salvar
novos_dados["CO_previsto"] = y_pred
novos_dados.to_csv("grafana_com_predicao.csv", index=False)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- CO
- CO_AE
- CO_WE
- temperatura
Feature names seen at fit time, yet now missing:
- e2sp_co
- e2sp_co_ae
- e2sp_co_we
- e2sp_temp
- pin_umid
